# 0. Preparing

In [1]:
from geoband.API import *

import math
from tqdm import tqdm
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.offline as py
import plotly.graph_objs as go
import plotly.tools as tls
import folium
import json
import geopandas as gpd
import geopy.distance



import pydeck as pdk
import shapely
import shapefile
import os
import re
import mapboxgl
from mapboxgl.viz import *
import seaborn as sns

import platform
import requests; from urllib.parse import urlparse

import warnings
warnings.filterwarnings(action='ignore') #경고 메시지 무시
from IPython.display import display #print가 아닌 display()로 연속 출력

/opt/app-root/lib/python3.6/site-packages/geopandas/_compat.py:91: UserWarning:

The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.



In [2]:
from tensorflow.keras.datasets.boston_housing import load_data
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [3]:
import mglearn
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV 
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle
from mlxtend.regressor import StackingRegressor
import keras.models
import keras.layers
import tensorflow as tf
from scipy.special import erfc
from sklearn.dummy import DummyRegressor
from sklearn.svm import SVR

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import train_test_split, cross_validate, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, RandomForestRegressor, VotingRegressor

from lightgbm import LGBMRegressor

import xgboost as xgb
from xgboost import XGBClassifier, XGBRegressor

import graphviz
import warnings


In [4]:
daejeon_data = gpd.read_file('daejeon_data.geojson')

> loss 함수정의

In [5]:
# 각각 root-mean-squared-error / mean abosolute error 구해주는 함수

def RMSE(y, pred):
    rmse = ( np.sum([x**2 for x in (np.array(y)-np.array(pred))]) / len(y) )**(1/2)
    return round(rmse, 4)
    
def MAE(y, pred):
    mae = np.sum([np.abs(x) for x in (np.array(y)-np.array(pred))]) / len(y)
    return round(mae, 4)

# 1. Epdo 지수 적용 사고건수 중 차대차 모형 생성

In [6]:
#교통량 있는 격자만 추출

data = daejeon_data[daejeon_data['isroad']==1]

In [7]:
#다른 종속변수들 및 인덱스용 변수 제외

data = data.drop(['acci_cnt','car_car','car_human','car_only','youth_attack','senior_attack','weak_attack',
                 'youth_victim','senior_victim','weak_victim','victim_na','acci_risk','acci_ratio',
                  'acci_ratio_ctc','acci_ratio_cth','acci_ratio_cal','isroad','geometry'],axis=1)

In [8]:
#스케일링ver / 원본으로 나누기

scaler = MinMaxScaler()

data_scaled = data.copy()


data_scaled.iloc[:, 1:43 ] = scaler.fit_transform(data_scaled.iloc[:, 1:43 ])

In [15]:
#불필요한 변수 삭제

data_car = data.drop(['acci_risk_cth','acci_risk_cal'],axis=1)        #차대사람, 차량단독 변수 제외
data_car_sc = data_scaled.drop(['acci_risk_cth','acci_risk_cal'],axis=1)        #차대사람, 차량단독 변수 제외

In [9]:
# train / test 데이터셋 분할

train, test = train_test_split(data_car, test_size=0.2, random_state=2021)

x_train = train.iloc[:, 1:42]
y_train = train.iloc[:, 43]
x_test = test.iloc[:, 1:42]
y_test = test.iloc[:, 43]

In [16]:
# train/test로 분할하지 않은 데이터셋

x_temp = data_car.iloc[:, 1:42]
y_temp = data_car.iloc[:, 43]

x_temp_sc = data_car_sc.iloc[:, 1:42]
y_temp_sc = data_car_sc.iloc[:, 43]

# 2. Machine Learning Model

- ###   RF regressor

In [11]:
rf = RandomForestRegressor(n_estimators = 100, random_state=123)
rf.fit(x_train, y_train)
features = x_train.columns.values

> 변수중요도 시각화

In [20]:
x, y = (list(x) for x in zip(*sorted(zip(rf.feature_importances_,features), reverse=False)))

trace = go.Bar(x=x, y=y, marker = dict(
                                color = x,
                                colorscale = 'Viridis',
                                reversescale = True),
                              name = 'Random Forest Feature Importances',
                              orientation = 'h')

layout = dict(title = 'Barplot of feature importances', width = 900, height = 2000, 
              yaxis = dict(showgrid=False, showline = False, showticklabels = True, domain = [1,1]))

fig1 = go.Figure(data=[trace])
fig1['layout'].update(layout)
py.iplot(fig1, filename='plots')

교통량 합계와 차량등의 중요도가 높음

#### 모델 성능평가

k-fold 교차검증

In [13]:
k_fold = KFold(n_splits=5, shuffle=True, random_state=2021)

In [14]:
from sklearn.metrics import make_scorer


def RMSE(pred, y):
    rmse = ( np.sum([x**2 for x in (np.array(y)-np.array(pred))]) / len(y) )**(1/2)
    return round(rmse, 4)
    
def MAE(pred, y):
    mae = np.sum([np.abs(x) for x in (np.array(y)-np.array(pred))]) / len(y)
    return round(mae, 4)


rmse_scorer = make_scorer(RMSE)
mae_scorer = make_scorer(MAE)

#### 원본 데이터 스코어

In [75]:
rmse_score = cross_val_score(rf, x_temp, y_temp, cv=k_fold, scoring = rmse_scorer)
mae_score = cross_val_score(rf, x_temp, y_temp, cv=k_fold, scoring = mae_scorer)

In [76]:
rmse_score = rmse_score.mean()
mae_score = mae_score.mean()

print('rmse: score={0:.5f}'.format(rmse_score))
print('mae: score={0:.5f}'.format(mae_score))

rmse: score=3.15572
mae: score=1.35036


#### 스케일 데이터 스코어

In [82]:
rmse_score = cross_val_score(rf, x_temp_sc, y_temp_sc, cv=k_fold, scoring = rmse_scorer)
mae_score = cross_val_score(rf, x_temp_sc, y_temp_sc, cv=k_fold, scoring = mae_scorer)

In [83]:
rmse_score = rmse_score.mean()
mae_score = mae_score.mean()

print('rmse: score={0:.5f}'.format(rmse_score))
print('mae: score={0:.5f}'.format(mae_score))

rmse: score=3.14444
mae: score=1.34148


- ### LGBM regressor

In [22]:
lgbm = LGBMRegressor(num_iterations = 200, random_state=2021)

#### 원본 데이터 스코어

In [87]:
rmse_score = cross_val_score(lgbm, x_temp, y_temp, cv=k_fold, scoring = rmse_scorer)
mae_score = cross_val_score(lgbm, x_temp, y_temp, cv=k_fold, scoring = mae_scorer)

In [88]:
rmse_score = rmse_score.mean()
mae_score = mae_score.mean()

print('rmse: score={0:.5f}'.format(rmse_score))
print('mae: score={0:.5f}'.format(mae_score))

rmse: score=3.12622
mae: score=1.30864


#### 스케일 데이터 스코어

In [26]:
rmse_score = cross_val_score(lgbm, x_temp_sc, y_temp_sc, cv=k_fold, scoring = rmse_scorer)
mae_score = cross_val_score(lgbm, x_temp_sc, y_temp_sc, cv=k_fold, scoring = mae_scorer)

In [27]:
rmse_score = rmse_score.mean()
mae_score = mae_score.mean()

print('rmse: score={0:.5f}'.format(rmse_score))
print('mae: score={0:.5f}'.format(mae_score))

rmse: score=3.12622
mae: score=1.30864


- ### XGB regressor

In [89]:
xgb = XGBRegressor(booster = 'gbtree',n_estimators = 200, random_state=2021)

#### 원본 데이터 스코어

In [90]:
rmse_score = cross_val_score(xgb, x_temp, y_temp, cv=k_fold, scoring = rmse_scorer)
mae_score = cross_val_score(xgb, x_temp, y_temp, cv=k_fold, scoring = mae_scorer)

In [91]:
rmse_score = rmse_score.mean()
mae_score = mae_score.mean()

print('rmse: score={0:.5f}'.format(rmse_score))
print('mae: score={0:.5f}'.format(mae_score))

rmse: score=3.27824
mae: score=1.39372


#### 스케일 데이터 스코어

In [107]:
rmse_score = cross_val_score(xgb, x_temp_sc, y_temp_sc, cv=k_fold, scoring = rmse_scorer)
mae_score = cross_val_score(xgb, x_temp_sc, y_temp_sc, cv=k_fold, scoring = mae_scorer)

In [108]:
rmse_score = rmse_score.mean()
mae_score = mae_score.mean()

print('rmse: score={0:.5f}'.format(rmse_score))
print('mae: score={0:.5f}'.format(mae_score))

rmse: score=3.27922
mae: score=1.39388


- ### knn

In [101]:
knn = KNeighborsRegressor()

In [102]:
rmse_score = cross_val_score(knn, x_temp, y_temp, cv=k_fold, scoring = rmse_scorer)
mae_score = cross_val_score(knn, x_temp, y_temp, cv=k_fold, scoring = mae_scorer)

In [103]:
rmse_score = rmse_score.mean()
mae_score = mae_score.mean()

print('rmse: score={0:.5f}'.format(rmse_score))
print('mae: score={0:.5f}'.format(mae_score))

rmse: score=4.09636
mae: score=1.72292


스케일

In [111]:
rmse_score = cross_val_score(knn, x_temp_sc, y_temp_sc, cv=k_fold, scoring = rmse_scorer)
mae_score = cross_val_score(knn, x_temp_sc, y_temp_sc, cv=k_fold, scoring = mae_scorer)

In [112]:
rmse_score = rmse_score.mean()
mae_score = mae_score.mean()

print('rmse: score={0:.5f}'.format(rmse_score))
print('mae: score={0:.5f}'.format(mae_score))

rmse: score=3.51486
mae: score=1.39640


- ### 릿지

In [104]:
rid = Ridge()

In [105]:
rmse_score = cross_val_score(rid, x_temp, y_temp, cv=k_fold, scoring = rmse_scorer)
mae_score = cross_val_score(rid, x_temp, y_temp, cv=k_fold, scoring = mae_scorer)

In [106]:
rmse_score = rmse_score.mean()
mae_score = mae_score.mean()

print('rmse: score={0:.5f}'.format(rmse_score))
print('mae: score={0:.5f}'.format(mae_score))

rmse: score=3.35202
mae: score=1.63068


스케일

In [109]:
rmse_score = cross_val_score(rid, x_temp_sc, y_temp_sc, cv=k_fold, scoring = rmse_scorer)
mae_score = cross_val_score(rid, x_temp_sc, y_temp_sc, cv=k_fold, scoring = mae_scorer)

In [110]:
rmse_score = rmse_score.mean()
mae_score = mae_score.mean()

print('rmse: score={0:.5f}'.format(rmse_score))
print('mae: score={0:.5f}'.format(mae_score))

rmse: score=3.29762
mae: score=1.60796


- ### SVR

In [11]:
svr_lin = SVR(kernel = 'linear')
svr_poly = SVR(kernel = 'poly')
svr_rbf = SVR(kernel = 'rbf')
svr_sig = SVR(kernel = 'sigmoid')

ploy 커넬

In [35]:
rmse_score_poly = cross_val_score(svr_poly, x_temp, y_temp, cv=k_fold, scoring = rmse_scorer)
mae_score_poly = cross_val_score(svr_poly, x_temp, y_temp, cv=k_fold, scoring = mae_scorer)

In [36]:
rmse_score_poly = rmse_score_poly.mean()
mae_score_poly = mae_score_poly.mean()

print('rmse: score={0:.5f}'.format(rmse_score_poly))
print('mae: score={0:.5f}'.format(mae_score_poly))

rmse: score=4.38074
mae: score=1.54948


rbf 커넬

In [17]:
rmse_score_rbf = cross_val_score(svr_rbf, x_temp, y_temp, cv=k_fold, scoring = rmse_scorer)
mae_score_rbf = cross_val_score(svr_rbf, x_temp, y_temp, cv=k_fold, scoring = mae_scorer)

In [18]:
rmse_score_rbf = rmse_score_rbf.mean()
mae_score_rbf = mae_score_rbf.mean()

print('rmse: score={0:.5f}'.format(rmse_score_rbf))
print('mae: score={0:.5f}'.format(mae_score_rbf))

rmse: score=4.20186
mae: score=1.54114


# 3. Deep Learning Model

- ### 다층 퍼셉트론(keras sequential) 

In [11]:
# activation function 정의: selu function

alpha_0_1 = -np.sqrt(2/np.pi) / ( erfc(1/np.sqrt(2)) * np.exp(1/2) -1 )
scale_0_1 = ( 1 - erfc(1/np.sqrt(2)) * np.sqrt(np.e)) * np.sqrt(2*np.pi) * ( 2 * erfc(np.sqrt(2))*np.e**2 + 
                                                                           np.pi*erfc(1/np.sqrt(2))**2*np.e -
                                                                           2*(2+np.pi)*erfc(1/np.sqrt(2))*np.sqrt(np.e) +
                                                                           np.pi + 2 )**(-1/2)

def selu(z, scale = scale_0_1, alpha = alpha_0_1) :
    return scale * tf.where(z>=0.0, z, alpha*tf.nn.elu(z))

In [23]:
def build_model() :
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(30, activation=selu, input_shape=(x_train.shape[1],)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dense(15, activation=selu))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dense(1))
    opt = tf.compat.v1.train.GradientDescentOptimizer(learning_rate=0.01)
    model.compile(optimizer=opt, loss='mse', metrics=['mae'])
    return model

In [44]:
k_fold = KFold(n_splits=5, shuffle=True, random_state=2021)

In [21]:
from keras.wrappers.scikit_learn import KerasRegressor

In [24]:
model = KerasRegressor(build_fn=build_model, epochs=150, batch_size=10, verbose=0)

In [49]:
rmse_score = cross_val_score(model, x_temp, y_temp, cv=k_fold, scoring = rmse_scorer)
mae_score = cross_val_score(model, x_temp, y_temp, cv=k_fold, scoring = mae_scorer)

In [50]:
rmse_score = rmse_score.mean()
mae_score = mae_score.mean()

print('rmse: score={0:.5f}'.format(rmse_score))
print('mae: score={0:.5f}'.format(mae_score))

rmse: score=263.79558
mae: score=10.92802


# 4. 결과


> 딥러닝모델의 경우, 오버피팅에 민감한 특성이 있기 때문에 매우 좋지 않은 결과를 볼 수 있다.

> 머신러닝 모델 사이에서는 LGBM의 성능이 가장 좋지만 다른 모델들과 큰 차이를 볼 수는 없었다.

> __딥러닝과 머신러닝 모델 모두 눈에 띄게 좋은 예측력을 보여주지 못했고, 결과에 대한 해석이 어려워 교통사고에 영향을 주는 요인을 파악해야 하는 우리 분석의 방향과 맞지 않다고 생각했다.__